In [ ]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import os
from sklearn.preprocessing import OneHotEncoder

In [3]:
df= pd.read_csv(r'..\Datasets\mini_telcom.csv')
df.head()

,Gender,TT_gt_100,Response
0,male,y,not bought
1,male,n,not bought
2,female,n,not bought
3,female,n,not bought
4,male,n,not bought


In [4]:
#Apply one-hot encoding to the categorical variables in mini_telcom.csv
ohe=OneHotEncoder(sparse=False, drop='first').set_output(transform='pandas')
ohe.fit_transform(df)

,Gender _male,TT_gt_100_y,Response _not bought
0,1.0,1.0,1.0
1,1.0,0.0,1.0
2,0.0,0.0,1.0
3,0.0,0.0,1.0
4,1.0,0.0,1.0
5,1.0,0.0,1.0
6,1.0,1.0,0.0
7,0.0,1.0,0.0
8,0.0,0.0,0.0
9,0.0,1.0,0.0


#Apply one-hot encoding to the categorical variables in Housing.csv

In [5]:
house_df = pd.read_csv(r'..\Datasets\Housing.csv')
house_df.head()

,price,lotsize,bedrooms,bathrms,stories,driveway,recroom,fullbase,gashw,airco,garagepl,prefarea
0,42000.0,5850,3,1,2,yes,no,yes,no,no,1,no
1,38500.0,4000,2,1,1,yes,no,no,no,no,0,no
2,49500.0,3060,3,1,1,yes,no,no,no,no,0,no
3,60500.0,6650,3,1,2,yes,yes,no,no,no,0,no
4,61000.0,6360,2,1,1,yes,no,no,no,no,0,no


In [6]:
ohe=OneHotEncoder(sparse=False, drop='first').set_output(transform='pandas')
ohe.fit_transform(house_df[['driveway','recroom','fullbase','gashw','airco','prefarea']]).head()

,driveway_yes,recroom_yes,fullbase_yes,gashw_yes,airco_yes,prefarea_yes
0,1.0,0.0,1.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0
3,1.0,1.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0


make column transformations to the dataset

In [7]:
from sklearn.compose import make_column_transformer
ct=make_column_transformer((ohe, ['driveway','recroom','fullbase','gashw','airco','prefarea']), remainder='passthrough',verbose_feature_names_out=False)
ct

ColumnTransformer(remainder='passthrough',
                  transformers=[('onehotencoder',
                                 OneHotEncoder(drop='first', sparse=False,
                                               sparse_output=False),
                                 ['driveway', 'recroom', 'fullbase', 'gashw',
                                  'airco', 'prefarea'])],
                  verbose_feature_names_out=False)

In [10]:
ct.set_output(transform='pandas')
ohe_df=ct.fit_transform(house_df)

ohe_df.head()

,driveway_yes,recroom_yes,fullbase_yes,gashw_yes,airco_yes,prefarea_yes,price,lotsize,bedrooms,bathrms,stories,garagepl
0,1.0,0.0,1.0,0.0,0.0,0.0,42000.0,5850,3,1,2,1
1,1.0,0.0,0.0,0.0,0.0,0.0,38500.0,4000,2,1,1,0
2,1.0,0.0,0.0,0.0,0.0,0.0,49500.0,3060,3,1,1,0
3,1.0,1.0,0.0,0.0,0.0,0.0,60500.0,6650,3,1,2,0
4,1.0,0.0,0.0,0.0,0.0,0.0,61000.0,6360,2,1,1,0


Using `make_colum_selector()`

rather than passing each column to the ohe we get the columns with object type using make_column_selector

In [15]:
from sklearn.compose import make_column_selector
ct1=make_column_transformer((ohe, make_column_selector(dtype_include='object')), remainder='passthrough',verbose_feature_names_out=False)
ct1

ColumnTransformer(remainder='passthrough',
                  transformers=[('onehotencoder',
                                 OneHotEncoder(drop='first', sparse=False,
                                               sparse_output=False),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x0000028FFDDF85E0>)],
                  verbose_feature_names_out=False)

In [16]:
ct1.set_output(transform='pandas')
ohe_df=ct1.fit_transform(house_df)

ohe_df.head()

,driveway_yes,recroom_yes,fullbase_yes,gashw_yes,airco_yes,prefarea_yes,price,lotsize,bedrooms,bathrms,stories,garagepl
0,1.0,0.0,1.0,0.0,0.0,0.0,42000.0,5850,3,1,2,1
1,1.0,0.0,0.0,0.0,0.0,0.0,38500.0,4000,2,1,1,0
2,1.0,0.0,0.0,0.0,0.0,0.0,49500.0,3060,3,1,1,0
3,1.0,1.0,0.0,0.0,0.0,0.0,60500.0,6650,3,1,2,0
4,1.0,0.0,0.0,0.0,0.0,0.0,61000.0,6360,2,1,1,0


Using pipeline to do the column transform anf training the model

In [19]:
from sklearn.pipeline import Pipeline
lr=LinearRegression()
pipe1=Pipeline(steps=[('transform', ct1), ('lr', lr)])
x,y=house_df.drop(columns=['price']), house_df['price']

pipe1.fit(x,y)

Pipeline(steps=[('transform',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(drop='first',
                                                                sparse=False,
                                                                sparse_output=False),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x0000028FFDDF85E0>)],
                                   verbose_feature_names_out=False)),
                ('lr', LinearRegression())])

In [20]:
lr.coef_

array([6.68777889e+03, 4.51128383e+03, 5.45238554e+03, 1.28314063e+04,
       1.26328904e+04, 9.36951324e+03, 3.54630297e+00, 1.83200347e+03,
       1.43355585e+04, 6.55694571e+03, 4.24482900e+03])